In [1]:
%env CUDA_VISIBLE_DEVICES=0
%env XLA_PYTHON_CLIENT_PREALLOCATE=false

env: CUDA_VISIBLE_DEVICES=0
env: XLA_PYTHON_CLIENT_PREALLOCATE=false


In [2]:
import brax
from brax import jumpy as jp
from brax.envs import env
import jax.numpy as jnp
import jax
from brax.envs import wrappers
import numpy as np
from brax.envs import humanoid

In [3]:
key = jax.random.PRNGKey(777)
key_envs = jax.random.split(key, 1)

In [4]:
from es_map.my_brax_envs import brax_envs
from es_map import jax_evaluate
from es_map import jax_es_update

In [13]:
env = brax_envs.FelxibleAnt() 
env = wrappers.VectorWrapper(env, batch_size=10)
env = wrappers.VectorGymWrapper(env)


In [20]:
obs = env.reset()
action = env.action_space.sample()
obs,rew,done,info = env.step(action)

In [14]:
state = env.reset(key)

In [9]:
action = np.random.randn(env.action_size)

In [10]:
state,p_info = env._step(state,action)

In [11]:
state = env.step(state,action)

In [12]:
state.info["contact_cum"]

DeviceArray([0., 0., 1., 1.], dtype=float32)

In [5]:
config = {
    "env_name" : "ant", #  ant, walker, hopper, halfcheetah, humanoid, ant_omni, humanoid_omni
    "env_mode" : "DIRECTIONAL_CONTACT",
    "episode_max_length" : 20,
    "env_deterministic" : True, 


    "ALGORITHM_TYPE" : "PLAIN_ES",   # "PLAIN_ES", "MAP_ES", "MAP_GA"

    "ES_NUM_GENERATIONS" : 500,  # was 1000
    "ES_popsize" : 50,
    "ES_sigma" : 0.02,
    "ES_OPTIMIZER_TYPE" : "ADAM",
    "ES_lr" : 0.01,
    "ES_L2_COEFF" : 0.005,  

    "ES_CENTRAL_NUM_EVALUATIONS" : 50, # How many times central individual evaluated (pointless for deterministic...)

    "ES_UPDATES_MODES_TO_USE" : ["fitness"], # "fitness","evo_var","evo_ent","innovation",...

    "NOVELTY_CALCULATION_NUM_NEIGHBORS" : 10,
    "ENTROPY_CALCULATION_KERNEL_BANDWIDTH" : 0.25, # maybe this should be different for final pos and foot contacts...

    "CHECKPOINT_FREQUENCY" : 100,

    "PLAIN_ES_TEST_ELITE_MAPPING_PERFORMANCE" : True,
    "PLAIN_ES_TEST_ELITE_MAPPING_FREQUENCY" : 1, # try filling the map every n steps

    "BMAP_type_and_metrics" : {
        "type" : "single_map",
        "metrics" : ["eval_fitness"],
    }
}

env_name = config["env_name"]
env_mode = config["env_mode"]
population_size = config["ES_popsize"]
    
env = jax_evaluate.create_env(env_name,env_mode,population_size,config["ES_CENTRAL_NUM_EVALUATIONS"],config["episode_max_length"])
model = jax_evaluate.create_MLP_model(env.observation_space.shape[1],env.action_space.shape[1])

# setup batched functions
batch_model_apply_fn = jax.jit(jax.vmap(model.apply))                                   # This does not work (hangs) if not jitted  WTF???
batch_vec_to_params = jax.vmap(jax_evaluate.vec_to_params_tree,in_axes=[0, None,None])  # this does not work (hangs) when jitted    WTF???
calculate_novelty_fn = jax_evaluate.get_calculate_novelty_fn(k=config["NOVELTY_CALCULATION_NUM_NEIGHBORS"])
batch_calculate_novelty_fn = jax.jit(jax.vmap(calculate_novelty_fn,in_axes=[0, None]))

key, key_init_model = jax.random.split(key, 2)

initial_model_params = model.init(key_init_model)
vec,shapes,indicies = jax_evaluate.params_tree_to_vec(initial_model_params)

current_params = np.array(vec)

optimizer_state = None
observation_stats = {                  # this is a single obs stats to keep track of during the whole experiment. 
    "sum" : np.zeros(env.observation_space.shape[1]),       # This is always expanded, and always used to calculate the current mean and std
    "sumsq" : np.zeros(env.observation_space.shape[1]),
    "count" : 0,
}
    

In [6]:
key, key_create_pop = jax.random.split(key, 2)
all_params,perturbations = jax_es_update.jax_es_create_population(current_params,key_create_pop,
                                                                  popsize=population_size,
                                                                  eval_batch_size=config["ES_CENTRAL_NUM_EVALUATIONS"],sigma=config["ES_sigma"])
all_model_params = batch_vec_to_params(all_params,shapes,indicies)

rollout_results,new_obs_stats = jax_evaluate.rollout_episodes(env,all_model_params,observation_stats,config,
                                                                    batch_model_apply_fn)


contact_cum before  (100, 4)
contact_cum (100, 4)
current_foot_contacts (100, 4)
foot_contacts (100, 4)
contact_cum (100, 4)
current_foot_contacts (100, 4)
foot_contacts (100, 4)
contact_cum (100, 4)
current_foot_contacts (100, 4)
foot_contacts (100, 4)
contact_cum (100, 4)
current_foot_contacts (100, 4)
foot_contacts (100, 4)
contact_cum (100, 4)
current_foot_contacts (100, 4)
foot_contacts (100, 4)
contact_cum (100, 4)
current_foot_contacts (100, 4)
foot_contacts (100, 4)
contact_cum (100, 4)
current_foot_contacts (100, 4)
foot_contacts (100, 4)
contact_cum (100, 4)
current_foot_contacts (100, 4)
foot_contacts (100, 4)
contact_cum (100, 4)
current_foot_contacts (100, 4)
foot_contacts (100, 4)
contact_cum (100, 4)
current_foot_contacts (100, 4)
foot_contacts (100, 4)
contact_cum (100, 4)
current_foot_contacts (100, 4)
foot_contacts (100, 4)
contact_cum (100, 4)
current_foot_contacts (100, 4)
foot_contacts (100, 4)
contact_cum (100, 4)
current_foot_contacts (100, 4)
foot_contacts (100,

In [7]:
rollout_results

{'control_cost_fitness': DeviceArray([16.425243 , 17.780113 , 15.182276 , 16.437584 , 17.868607 ,
              17.255932 , 14.734159 , 17.102167 , 15.195849 , 16.9831   ,
              16.724293 , 17.016851 , 16.006826 , 17.574345 , 16.587461 ,
              18.240688 , 16.599077 , 16.239573 , 14.471835 , 14.192604 ,
              16.248474 , 17.08618  , 12.927573 , 16.192135 , 17.482765 ,
              17.785603 , 12.424889 , 17.12631  , 16.853138 , 18.009598 ,
              17.860216 , 17.309444 , 13.948363 , 17.541754 , 17.848787 ,
              11.898574 , 16.827703 , 17.675093 , 16.870348 , 17.2975   ,
              17.69554  , 17.668644 , 18.136366 , 17.452635 , 17.840122 ,
              16.823275 , 17.573593 , 18.236238 , 16.622398 , 16.749699 ,
              17.071178 , 17.14509  , 17.70788  , 15.861824 , 17.46125  ,
              17.395617 , 17.114449 , 15.630429 , 17.968513 , 17.334538 ,
              16.108372 , 17.605759 , 17.338816 , 16.74856  , 18.243523 ,
              

In [ ]:
obs = env.reset()

In [9]:
env._state.info.keys()

dict_keys(['bd_final_pos', 'bd_foot_contacts', 'contact_cum', 'first_obs', 'first_qp', 'fitness_control_cost_and_survive', 'fitness_directional_distance', 'fitness_distance_walked', 'fitness_forward', 'steps', 'truncation'])

In [8]:
env._state.info["fitness_control_cost_and_survive"].shape

(1050,)

In [13]:
control_cost_fitness = jnp.zeros_like(env._state.info["fitness_control_cost_and_survive"])
active_episode = jnp.ones_like(control_cost_fitness)
active_episode.shape

(1050,)

In [7]:
import numpy as np
x = np.arange(6).reshape(2, 3)
idx = [0, 1]
x[idx]


array([[0, 1, 2],
       [3, 4, 5]])

In [8]:
x[tuple(idx)]

1

In [9]:
a = jnp.array(np.arange(20))

In [11]:
jnp.take_along_axis(a,np.array([2,4]),axis=0)

DeviceArray([2, 4], dtype=int32)

In [27]:
def g(x,index):
    return jnp.take(x,index)


def fun(a):
    index = np.array([2,4])
    return g(a,index)
                    
ffun = jax.vmap(fun)
                    
a = jnp.array(np.arange(20).reshape(4,-1))

In [28]:
ffun(a)

DeviceArray([[ 2,  4],
             [ 7,  9],
             [12, 14],
             [17, 19]], dtype=int32)